In [5]:
from tensorflow.keras.models import load_model
import pickle
import tkinter as tk
from tkinter import scrolledtext, messagebox
import re
import string
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences


model = load_model('auto_correction_model.h5')
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))

In [6]:
def clean(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'\d+', ' ', text)  
    text = ''.join([i for i in text if i not in string.punctuation]) 
    return text


def correct_text():
    input_text = input_box.get("1.0", tk.END).strip()
    if not input_text:
        messagebox.showwarning("تنبيه", "الرجاء إدخال نص عربي.")
        return
    try:
        cleaned_text = clean(input_text)
        sequence = tokenizer.texts_to_sequences([cleaned_text])
        padded_sequence = pad_sequences(sequence, maxlen=394, padding='post')
        pred = model.predict(padded_sequence)
        pred_sequence = np.argmax(pred, axis=-1)
        corrected = tokenizer.sequences_to_texts(pred_sequence)
        output_box.delete("1.0", tk.END)
        output_box.insert(tk.END, corrected[0])
    except Exception as e:
        messagebox.showerror("خطأ", f"حدث خطأ أثناء التصحيح:\n{e}")


def clear_text():
    input_box.delete("1.0", tk.END)
    output_box.delete("1.0", tk.END)


def exit_app():
    window.destroy()


window = tk.Tk()
window.title("مصحح النصوص العربية")
window.geometry("700x500")
window.configure(bg="#f7f7f7")


tk.Label(window, text="📝 أدخل النص هنا:", bg="#f7f7f7", font=("Arial", 14, "bold")).pack(pady=10)


input_box = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=80, height=8, font=("Arial", 12), fg="black")
input_box.pack(pady=5, padx=10)


button_frame = tk.Frame(window, bg="#f7f7f7")
button_frame.pack(pady=10)

tk.Button(button_frame, text="✔ تصحيح", command=correct_text, width=10, font=("Arial", 12), bg="#4CAF50", fg="white").grid(row=0, column=0, padx=5)
tk.Button(button_frame, text="🗑 مسح", command=clear_text, width=10, font=("Arial", 12), bg="#f39c12", fg="white").grid(row=0, column=1, padx=5)
tk.Button(button_frame, text="❌ خروج", command=exit_app, width=10, font=("Arial", 12), bg="#e74c3c", fg="white").grid(row=0, column=2, padx=5)


tk.Label(window, text="📘 النص المصحح:", bg="#f7f7f7", font=("Arial", 14, "bold")).pack(pady=10)


output_box = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=80, height=8, font=("Arial", 12), fg="darkgreen")
output_box.pack(pady=5, padx=10)


window.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
